In [1]:
import xarray as xr
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
data = xr.open_dataset("../ecco_data_minimal/southern_ocean.nc")
latitudes = data["latitude"].to_numpy()
infile = open("../sf/so/sf_density.pickle", "rb")
sf = pickle.load(infile); infile.close()
infile = open("../sf/so/density_range.pickle", "rb")
dr = pickle.load(infile); infile.close()

In [5]:
def plot_moc(lat):
    abyssal_cell = np.argmax(dr[lat] > 36.)
    psi = sf[lat, abyssal_cell:, :]
    if (psi == 0).all(): return
    d_0_ix = np.argmin(psi.mean(axis=-1))
    d_0 = dr[lat, abyssal_cell+d_0_ix]
    moc = psi[d_0_ix]

    outpath = f"so/{abs(latitudes[lat])}S_moc" 
    outfile = open(f"{outpath}.pickle", "wb")
    pickle.dump(moc, outfile); outfile.close()

    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(moc, color="red", linewidth=1)
    ax.set_ylabel("Abyssal MOC Strength [Sv]")
    ax.set_xlabel("Year")
    ax.set_xticks(np.arange(0, 324, 12), np.arange(1992, 2019), rotation=45)
    ax.set_title(f"{abs(latitudes[lat])}S: d$_0$={str(round(d_0, 2))}")
    plt.tight_layout()
    plot_title = f"plots_so/{abs(latitudes[lat])}S"
    plt.savefig(f"{plot_title}.png", dpi=400)
    plt.close()

In [6]:
for i in range(121): plot_moc(i)